In [2]:
from data_loader_df import *
import sys
import numpy
import pandas as pd

In [4]:
train_file_path = "./adult.data"
test_file_path = "./adult.test"

train, val, test = load_all_data(train_file_path, test_file_path, 0.1)
train_X, train_Y, val_X, val_Y, test_X, test_Y = load_all_data(train_file_path, test_file_path, 0.1, False)

TypeError: load_train_data() takes from 1 to 3 positional arguments but 4 were given

In [3]:
train

,age,fnlwgt,edunum,cgain,closs,hpw,income,work_ Federal-gov,work_ Local-gov,work_ Private,...,nation_ Portugal,nation_ Puerto-Rico,nation_ Scotland,nation_ South,nation_ Taiwan,nation_ Thailand,nation_ Trinadad&Tobago,nation_ United-States,nation_ Vietnam,nation_ Yugoslavia
24337,1.774289,-1.149839,-0.438122,-0.146733,-0.218780,-0.078120,<=50K,0,1,0,...,0,0,0,0,0,0,0,1,0,0
17049,0.866417,0.248540,-0.438122,-0.146733,-0.218780,-0.078120,>50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0
21016,-0.192767,-0.040987,-0.438122,-0.146733,-0.218780,0.754701,>50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2790,1.925601,0.646275,-0.046403,-0.146733,-0.218780,-0.078120,<=50K,0,0,1,...,0,0,0,0,0,0,0,0,0,0
13511,-0.798015,-1.423226,0.345316,-0.146733,-0.218780,1.587523,<=50K,0,0,0,...,0,0,0,0,0,0,0,1,0,0
29288,-1.478919,-0.841976,-0.438122,-0.146733,-0.218780,-0.078120,<=50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2161,-1.327607,-1.222057,-0.046403,-0.146733,-0.218780,-0.327966,<=50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0
12546,0.034201,-0.721840,1.128753,-0.146733,-0.218780,0.338291,>50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0
210,0.412481,-0.272551,-0.438122,-0.146733,-0.218780,-0.078120,<=50K,0,1,0,...,0,0,0,0,0,0,0,1,0,0
20261,2.076913,0.234142,-0.438122,-0.146733,-0.218780,-2.326738,<=50K,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [1]:
import sys
sys.executable

'/usr/local/opt/python/bin/python3.6'

In [5]:
train.shape[0] + val.shape[0]

30162

In [6]:
train_X.shape, train_Y.shape

((27158, 104), (27158,))

In [7]:
test_X.shape, test_Y.shape

((15060, 104), (15060,))

In [8]:
val_X.shape, val_Y.shape

((3004, 104), (3004,))

In [9]:
df

NameError: name 'df' is not defined

In [ ]:
valid_rate = 0.1
mask = np.random.rand(len(df)) < 1 - valid_rate
train_df, valid_df = df.iloc[mask, :], df.iloc[~mask, :]

In [ ]:
train_df

In [ ]:
valid_df

In [ ]:
train_labels = [1 if x == POS_STR else 0 for x in train_df['income']]

In [ ]:
train_labels

In [ ]:
train_df.drop('income', axis=1, inplace=True)